In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
import os

In [2]:
train = ImageDataGenerator(rescale=1/255)
validation = ImageDataGenerator(rescale=1/255)

In [3]:
### dilation and erosion
def thin_font(image): # Erosion
    image = cv2.bitwise_not(image)
    kernel = np.ones((2,2), np.uint8)
    image = cv2.erode(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return image

def thick_font(image): # Dilation
    image = cv2.bitwise_not(image)
    kernel = np.ones((2,2), np.uint8)
    image = cv2.dilate(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return image

In [4]:
# Function for color-based segmentation
def color_segmentation(image):
    # Convert image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define lower and upper bounds for red color
    lower_bound = np.array([0, 100, 100])
    upper_bound = np.array([10, 255, 255])
    
    # Threshold the HSV image to get a binary mask
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    
    # Apply morphological operations to clean up the mask (optional)
    kernel = np.ones((2, 2), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    return mask

In [5]:
training_defects = "./process_training/defects"
training_apples = "./process_training/apples"
validate_apples = "./process_validation/apples"
validate_defects = "./process_validation/defects"

# Output folder to save Color_segmentation images
output_train_defects = "./color_training/defects"
output_train_apples = "./color_training/apples"
output_valid_apples = "./color_validation/apples"
output_valid_defects = "./color_validation/defects"

# Process images in Training Defects
for filename in os.listdir(training_defects):
    img_path = os.path.join(training_defects, filename)
    img = cv2.imread(img_path)
#     thick_img = thick_font(img)
#     processed_img = thin_font(thick_img)
    processed_img = color_segmentation(img)
    output_path = os.path.join(output_train_defects, f'{filename}')
    cv2.imwrite(output_path, processed_img)

# Process images in Training Apples
for filename in os.listdir(training_apples):
    img_path = os.path.join(training_apples, filename)
    img = cv2.imread(img_path)
#     thick_img = thick_font(img)
#     processed_img = thin_font(thick_img)
    processed_img = color_segmentation(img)
    output_path = os.path.join(output_train_apples, f'{filename}')
    cv2.imwrite(output_path, processed_img)
    
# Process images in Validate Apples
for filename in os.listdir(validate_apples):
    img_path = os.path.join(validate_apples, filename)
    img = cv2.imread(img_path)
#     thick_img = thick_font(img)
#     processed_img = thin_font(thick_img)
    processed_img = color_segmentation(img)
    output_path = os.path.join(output_valid_apples, f'{filename}')
    cv2.imwrite(output_path, processed_img)
    
# Process images in Validate Defects
for filename in os.listdir(validate_defects):
    img_path = os.path.join(validate_defects, filename)
    img = cv2.imread(img_path)
#     thick_img = thick_font(img)
#     processed_img = thin_font(thick_img)
    processed_img = color_segmentation(img)
    output_path = os.path.join(output_valid_defects, f'{filename}')
    cv2.imwrite(output_path, processed_img)

In [5]:
train_dataset = train.flow_from_directory(
    './color_training/',
    target_size = (256, 256),
    batch_size = 10,
    class_mode = 'binary'
)

validation_dataset = validation.flow_from_directory(
    './color_validation/',
    target_size = (256, 256),
    batch_size = 10,
    class_mode = 'binary'
)

Found 456 images belonging to 2 classes.
Found 663 images belonging to 2 classes.


In [6]:
# To differentiate between classes
train_dataset.class_indices

{'apples': 0, 'defects': 1}

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(15, (3,3), activation='relu', input_shape=(256,256, 1)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(30, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(60, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')                                
])

In [8]:
# binary_crossentropy - binary classification (between the true labels and the predicted probabilities) 
# lr too hight may lead to instability or divergence - overshoot the minimum of the loss
# metric - track accuracy (the proportion of correctly classified samples)
model.compile(loss='binary_crossentropy',
              optimizer = RMSprop(learning_rate=0.001),
              metrics=['accuracy']
)

In [9]:
# steps_per_epoch = how many batches to process in each epoch
# lets take batch size as 100
# steps_per_epoch = total training data / batch size = 280 / 10 = 28
# the model will process 133 steps from the training dataset in each epoch
# epochs = train 30 times
model_fit = model.fit(
    train_dataset,
    steps_per_epoch = 28,
    epochs = 30,
    validation_data = validation_dataset
)

Epoch 1/30


28/28 [==============================] - 21s 659ms/step - loss: 0.7539 - accuracy: 0.6304 - val_loss: 0.7456 - val_accuracy: 0.6516
Epoch 2/30
28/28 [==============================] - 16s 550ms/step - loss: 0.5030 - accuracy: 0.7571 - val_loss: 0.8788 - val_accuracy: 0.6139
Epoch 3/30
28/28 [==============================] - 15s 561ms/step - loss: 0.4115 - accuracy: 0.8261 - val_loss: 0.8815 - val_accuracy: 0.6682
Epoch 4/30
28/28 [==============================] - 15s 538ms/step - loss: 0.2798 - accuracy: 0.8768 - val_loss: 1.0258 - val_accuracy: 0.6802
Epoch 5/30
28/28 [==============================] - 13s 473ms/step - loss: 0.2159 - accuracy: 0.9143 - val_loss: 1.1348 - val_accuracy: 0.6908
Epoch 6/30
28/28 [==============================] - 15s 537ms/step - loss: 0.1552 - accuracy: 0.9420 - val_loss: 1.6276 - val_accuracy: 0.7089
Epoch 7/30
28/28 [==============================] - 15s 558ms/step - loss: 0.0886 - accuracy: 0.9746 - val_loss: 1.8606 - val_accuracy: 0.66

In [11]:
model.save('./color_segmentation_apple.h5') # save performed morphology model

In [14]:
import cv2
import numpy as np
import tensorflow as tf

# Load the trained model
loaded_model = tf.keras.models.load_model('./color_segmentation_apple.h5')

# Function to preprocess the image for model input
def preprocess_img(edges):
    # Expand dimensions to match the expected input shape of the model
    color = np.expand_dims(edges, axis=0)
    return color

# Function for color-based segmentation
def color_segmentation(image):
    # Convert image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv = cv2.resize(hsv, (256,256))
    
    # Define lower and upper bounds for red color
    lower_bound = np.array([0, 100, 100])
    upper_bound = np.array([10, 255, 255])
    
    # Threshold the HSV image to get a binary mask
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    
    # Apply morphological operations to clean up the mask (optional)
    kernel = np.ones((2, 2), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    return mask

# Function to run prediction
def run_prediction(image_path):
    # Perform edge detection
    img = cv2.imread(image_path)
    images = color_segmentation(img)
    
    images = preprocess_img(images)
    
    # Make prediction using the loaded model
    prediction = loaded_model.predict(images)
    
    # Interpret the prediction
    print(prediction)
    print(prediction[0])
    print(prediction[0][0])
    if prediction[0][0] == 0:
        print("This is an apple")
    else:
        print("This is a defect apple")

# Example usage:
image_path = './testData/21.jpg'
run_prediction(image_path)

1/1 [==============================] - 0s 156ms/step
[[1.]]
[1.]
1.0
This is a defect apple
